<a href="https://colab.research.google.com/github/marcosvnespolo/Previsao_Inadimplencia/blob/main/Previs%C3%A3o_Inadimpl%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sobre o Projeto

Uma empresa de cartão de crédito forneceu um dataset que inclui dados financeiros e demográficos de uma amostra de 30.000 titulares de contas. Os dados estão apresentados em nível conta de crédio, ou seja, cada linha do dataset representa uma conta de um cliente distinto.
O objetivo do projeto é desenvolver um modelo que preveja se uma conta ficará inadimplente no próximo mês, de acordo com dados demográficos e históricos.

Os dados já foram preparados e estão disponíveis na pasta do projeto.

In [1]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Projetos Portfólio/Projetos DS com Python/Projeto 1/default_of_credit_card_clients__courseware_version_1_21_19.xls')

# Inspecionando os Dados

In [ ]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [ ]:
df.shape

(30000, 25)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          30000 non-null  object
 1   LIMIT_BAL                   30000 non-null  int64 
 2   SEX                         30000 non-null  int64 
 3   EDUCATION                   30000 non-null  int64 
 4   MARRIAGE                    30000 non-null  int64 
 5   AGE                         30000 non-null  int64 
 6   PAY_1                       30000 non-null  object
 7   PAY_2                       30000 non-null  int64 
 8   PAY_3                       30000 non-null  int64 
 9   PAY_4                       30000 non-null  int64 
 10  PAY_5                       30000 non-null  int64 
 11  PAY_6                       30000 non-null  int64 
 12  BILL_AMT1                   30000 non-null  int64 
 13  BILL_AMT2                   30000 non-null  in

In [ ]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

Aparentemente a coluna ID apresenta o nome de cada cliente na base de dados, porém, podemos confirmar esta informação através do método .nunique()

In [4]:
df['ID'].nunique()


29687

O número de ID's únicos é inferior ao número de linhas, desta forma, o ID não é um identificador único, como sugerido anteriormente, sabemos então que há alguma duplicação de ID's.

Porém, quantos ID's estão duplicados? É possível começar a responder essa pergunta através do método .value_counts().

In [5]:
contagem_id = df['ID'].value_counts()

In [9]:
contagem_id.head(10)

ad23fe5c-7b09    2
1fb3e3e6-a68d    2
89f8f447-fca8    2
7c9b7473-cc2f    2
90330d02-82d9    2
2a793ecf-05c6    2
75938fec-e5ec    2
7be61027-a493    2
a3a5c0fc-fdd6    2
b44b81b2-7789    2
Name: ID, dtype: int64

Podemos utilizar novamente o método .value_counts() para identificar quantos valores são únicos ou duplicados.

In [10]:
contagem_id.value_counts()

1    29374
2      313
Name: ID, dtype: int64